
# **Sentiment Analysis**



In [49]:
!pip install nltk

In [50]:
import pandas as pd
import numpy as np
from numpy import array,asarray,zeros
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from bs4 import BeautifulSoup
import nltk
import re
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
from sklearn import metrics

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [51]:
df = pd.read_csv("/content/drive/MyDrive/AIL303m _ AI1707 _ Lê Hồng Quang _ Project/IMDB Dataset.csv")

In [52]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [53]:
df.sentiment.value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [54]:
example_sentences = df['review'].loc[2]
example_sentences

'I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I\'d laughed at one of Woody\'s comedies in years (dare I say a decade?). While I\'ve never been impressed with Scarlet Johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than "Devil Wears Prada" and more interesting than "Superman" a great comedy to go see with friends.'

In [55]:
# 1. Xóa content HTML
soup = BeautifulSoup(example_sentences, "html.parser")
example_sentences = soup.get_text()
example_sentences

'I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.This was the most I\'d laughed at one of Woody\'s comedies in years (dare I say a decade?). While I\'ve never been impressed with Scarlet Johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.This may not be the crown jewel of his career, but it was wittier than "Devil Wears Prada" and more interesting than "Superman" a great comedy to go see with friends.'

In [56]:
# 2. Sử dụng Biểu thức chính quy để xóa mọi thứ
example_sentences = re.sub('\[[^]]*\]', ' ',example_sentences)
example_sentences = re.sub('[^a-zA-Z]', ' ',example_sentences)
example_sentences

'I thought this was a wonderful way to spend time on a too hot summer weekend  sitting in the air conditioned theater and watching a light hearted comedy  The plot is simplistic  but the dialogue is witty and the characters are likable  even the well bread suspected serial killer   While some may be disappointed when they realize this is not Match Point    Risk Addiction  I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love This was the most I d laughed at one of Woody s comedies in years  dare I say a decade    While I ve never been impressed with Scarlet Johanson  in this she managed to tone down her  sexy  image and jumped right into a average  but spirited young woman This may not be the crown jewel of his career  but it was wittier than  Devil Wears Prada  and more interesting than  Superman  a great comedy to go see with friends '

In [57]:
# 3. Chuyển qua từ thường(LowerCase)
example_sentences = example_sentences.lower()
example_sentences

'i thought this was a wonderful way to spend time on a too hot summer weekend  sitting in the air conditioned theater and watching a light hearted comedy  the plot is simplistic  but the dialogue is witty and the characters are likable  even the well bread suspected serial killer   while some may be disappointed when they realize this is not match point    risk addiction  i thought it was proof that woody allen is still fully in control of the style many of us have grown to love this was the most i d laughed at one of woody s comedies in years  dare i say a decade    while i ve never been impressed with scarlet johanson  in this she managed to tone down her  sexy  image and jumped right into a average  but spirited young woman this may not be the crown jewel of his career  but it was wittier than  devil wears prada  and more interesting than  superman  a great comedy to go see with friends '

In [58]:
# 4. Tách từ (split)
example_sentences = example_sentences.split()
example_sentences

['i',
 'thought',
 'this',
 'was',
 'a',
 'wonderful',
 'way',
 'to',
 'spend',
 'time',
 'on',
 'a',
 'too',
 'hot',
 'summer',
 'weekend',
 'sitting',
 'in',
 'the',
 'air',
 'conditioned',
 'theater',
 'and',
 'watching',
 'a',
 'light',
 'hearted',
 'comedy',
 'the',
 'plot',
 'is',
 'simplistic',
 'but',
 'the',
 'dialogue',
 'is',
 'witty',
 'and',
 'the',
 'characters',
 'are',
 'likable',
 'even',
 'the',
 'well',
 'bread',
 'suspected',
 'serial',
 'killer',
 'while',
 'some',
 'may',
 'be',
 'disappointed',
 'when',
 'they',
 'realize',
 'this',
 'is',
 'not',
 'match',
 'point',
 'risk',
 'addiction',
 'i',
 'thought',
 'it',
 'was',
 'proof',
 'that',
 'woody',
 'allen',
 'is',
 'still',
 'fully',
 'in',
 'control',
 'of',
 'the',
 'style',
 'many',
 'of',
 'us',
 'have',
 'grown',
 'to',
 'love',
 'this',
 'was',
 'the',
 'most',
 'i',
 'd',
 'laughed',
 'at',
 'one',
 'of',
 'woody',
 's',
 'comedies',
 'in',
 'years',
 'dare',
 'i',
 'say',
 'a',
 'decade',
 'while',
 'i

In [59]:
# 5. Xử lí từ dừng(stopword)
example_sentences = [word for word in example_sentences if not word in set(stopwords.words('english'))]
example_sentences

['thought',
 'wonderful',
 'way',
 'spend',
 'time',
 'hot',
 'summer',
 'weekend',
 'sitting',
 'air',
 'conditioned',
 'theater',
 'watching',
 'light',
 'hearted',
 'comedy',
 'plot',
 'simplistic',
 'dialogue',
 'witty',
 'characters',
 'likable',
 'even',
 'well',
 'bread',
 'suspected',
 'serial',
 'killer',
 'may',
 'disappointed',
 'realize',
 'match',
 'point',
 'risk',
 'addiction',
 'thought',
 'proof',
 'woody',
 'allen',
 'still',
 'fully',
 'control',
 'style',
 'many',
 'us',
 'grown',
 'love',
 'laughed',
 'one',
 'woody',
 'comedies',
 'years',
 'dare',
 'say',
 'decade',
 'never',
 'impressed',
 'scarlet',
 'johanson',
 'managed',
 'tone',
 'sexy',
 'image',
 'jumped',
 'right',
 'average',
 'spirited',
 'young',
 'woman',
 'may',
 'crown',
 'jewel',
 'career',
 'wittier',
 'devil',
 'wears',
 'prada',
 'interesting',
 'superman',
 'great',
 'comedy',
 'go',
 'see',
 'friends']

In [60]:
# 6. Stemming/Lemmatization
ps = PorterStemmer()
ps_example_sentences = [ps.stem(word) for word in example_sentences]
ps_example_sentences

['thought',
 'wonder',
 'way',
 'spend',
 'time',
 'hot',
 'summer',
 'weekend',
 'sit',
 'air',
 'condit',
 'theater',
 'watch',
 'light',
 'heart',
 'comedi',
 'plot',
 'simplist',
 'dialogu',
 'witti',
 'charact',
 'likabl',
 'even',
 'well',
 'bread',
 'suspect',
 'serial',
 'killer',
 'may',
 'disappoint',
 'realiz',
 'match',
 'point',
 'risk',
 'addict',
 'thought',
 'proof',
 'woodi',
 'allen',
 'still',
 'fulli',
 'control',
 'style',
 'mani',
 'us',
 'grown',
 'love',
 'laugh',
 'one',
 'woodi',
 'comedi',
 'year',
 'dare',
 'say',
 'decad',
 'never',
 'impress',
 'scarlet',
 'johanson',
 'manag',
 'tone',
 'sexi',
 'imag',
 'jump',
 'right',
 'averag',
 'spirit',
 'young',
 'woman',
 'may',
 'crown',
 'jewel',
 'career',
 'wittier',
 'devil',
 'wear',
 'prada',
 'interest',
 'superman',
 'great',
 'comedi',
 'go',
 'see',
 'friend']

In [61]:
# 7. Lemmatization/WordNetLemmatizer
lem = WordNetLemmatizer()
example_sentences = [lem.lemmatize(word) for word in example_sentences]
example_sentences

['thought',
 'wonderful',
 'way',
 'spend',
 'time',
 'hot',
 'summer',
 'weekend',
 'sitting',
 'air',
 'conditioned',
 'theater',
 'watching',
 'light',
 'hearted',
 'comedy',
 'plot',
 'simplistic',
 'dialogue',
 'witty',
 'character',
 'likable',
 'even',
 'well',
 'bread',
 'suspected',
 'serial',
 'killer',
 'may',
 'disappointed',
 'realize',
 'match',
 'point',
 'risk',
 'addiction',
 'thought',
 'proof',
 'woody',
 'allen',
 'still',
 'fully',
 'control',
 'style',
 'many',
 'u',
 'grown',
 'love',
 'laughed',
 'one',
 'woody',
 'comedy',
 'year',
 'dare',
 'say',
 'decade',
 'never',
 'impressed',
 'scarlet',
 'johanson',
 'managed',
 'tone',
 'sexy',
 'image',
 'jumped',
 'right',
 'average',
 'spirited',
 'young',
 'woman',
 'may',
 'crown',
 'jewel',
 'career',
 'wittier',
 'devil',
 'wear',
 'prada',
 'interesting',
 'superman',
 'great',
 'comedy',
 'go',
 'see',
 'friend']

In [62]:
# 8. Nối lại các từ(join)
example_sentences = ' '.join(example_sentences)
example_sentences

'thought wonderful way spend time hot summer weekend sitting air conditioned theater watching light hearted comedy plot simplistic dialogue witty character likable even well bread suspected serial killer may disappointed realize match point risk addiction thought proof woody allen still fully control style many u grown love laughed one woody comedy year dare say decade never impressed scarlet johanson managed tone sexy image jumped right average spirited young woman may crown jewel career wittier devil wear prada interesting superman great comedy go see friend'

In [63]:
# Khởi tạo một kho(corpus) để chứa dữ liệu câu
corpus = []
corpus.append(example_sentences)

In [64]:
#sử dụng TfidfVectorizer (TF-IDF) & Tính (TF * IDF) của mô hình túi từ
tfidf_vecto = TfidfVectorizer()
sentences_tfidf_vecto = tfidf_vecto.fit_transform(corpus)

In [65]:
#Chia tập dữ liệu
x_train, x_test, y_train, y_test= train_test_split(df['review'], df['sentiment'], test_size=0.25, random_state=42)
y_train = (y_train.replace({'positive': 1, 'negative': 0})).values
y_test  = (y_test.replace({'positive': 1, 'negative': 0})).values

In [66]:
#Tiền xử lí dữ liệu cho tập dữ liệu
corpus_train = []
for i in range(x_train.shape[0]):
    soup = BeautifulSoup(x_train.iloc[i], "html.parser")
    review = soup.get_text()
    review = re.sub('\[[^]]*\]', ' ', review)
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    review = [word for word in review if not word in set(stopwords.words('english'))]
    lem = WordNetLemmatizer()
    review = [lem.lemmatize(word) for word in review]
    review = ' '.join(review)
    corpus_train.append(review)

<ipython-input-66-c0f089be8ff0>:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(x_train.iloc[i], "html.parser")


In [67]:
corpus_test  = []
for j in range(x_test.shape[0]):
    soup = BeautifulSoup(x_test.iloc[j], "html.parser")
    review = soup.get_text()
    review = re.sub('\[[^]]*\]', ' ', review)
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    review = [word for word in review if not word in set(stopwords.words('english'))]
    lem = WordNetLemmatizer()
    review = [lem.lemmatize(word) for word in review]
    review = ' '.join(review)
    corpus_test.append(review)

<ipython-input-67-14c3d01403f6>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(x_test.iloc[j], "html.parser")


In [68]:
# Vectơ hóa bằng kỹ thuật TF-IDF
tfidf_vecto = TfidfVectorizer(ngram_range=(1, 3))
tfidf_vecto_train = tfidf_vecto.fit_transform(corpus_train)
tfidf_vecto_test = tfidf_vecto.transform(corpus_test)

**TRIỂN KHAI MÔ HÌNH**

In [69]:
LSVC = LinearSVC(C=0.5, random_state=42)
LSVC.fit(tfidf_vecto_train, y_train)
du_doan = LSVC.predict(tfidf_vecto_test)

In [70]:
# Hiển thị bảng các chỉ số đánh giá mô hình và confusion matrix
print("Classification report of Linear SVC: \n",classification_report(y_test,du_doan, target_names = ['Positive','Negative']))
print("Confusion Matrix of Linear SVC: \n",confusion_matrix(y_test,du_doan))

dochinhxac = metrics.accuracy_score(y_test,du_doan)
print("Accuracy: " +str('{:04.2f}'.format(dochinhxac *100)) + "%")

f1 = metrics.f1_score(y_test,du_doan)
print("f1-score: \n"+str('{:04.2f}'.format(f1 *100)) + "%")

Classification report of Linear SVC: 
               precision    recall  f1-score   support

    Positive       0.91      0.89      0.90      6157
    Negative       0.89      0.92      0.91      6343

    accuracy                           0.90     12500
   macro avg       0.90      0.90      0.90     12500
weighted avg       0.90      0.90      0.90     12500

Confusion Matrix of Linear SVC: 
 [[5467  690]
 [ 524 5819]]
Accuracy: 90.29%
f1-score: 
90.55%


## Nhận xét
Với giá trị C = 0.5, chỉ số độ chính xác cũng như các chỉ số khác có vẻ khá cao chứng tỏ mô hình hoạt động khá tốt. Vậy khi điều chỉnh siêu tham số này có thể giúp mô hình hoạt động tốt hơn không?

## Điều chỉnh siêu tham số C:
### C = 1

In [71]:
LSVC = LinearSVC(C=1, random_state=42)
LSVC.fit(tfidf_vecto_train, y_train)
du_doan = LSVC.predict(tfidf_vecto_test)

In [72]:
print("Classification report: \n",classification_report(y_test,du_doan, target_names = ['Positive','Negative']))
print("Confusion Matrix: \n",confusion_matrix(y_test,du_doan))
dochinhxac = metrics.accuracy_score(y_test,du_doan)
print("Accuracy with C = 1: " +str('{:04.2f}'.format(dochinhxac *100)) + "%")

f1 = metrics.f1_score(y_test,du_doan)
print("f1-score : \n"+str('{:04.2f}'.format(f1 *100)) + "%")

Classification report: 
               precision    recall  f1-score   support

    Positive       0.92      0.89      0.90      6157
    Negative       0.90      0.92      0.91      6343

    accuracy                           0.91     12500
   macro avg       0.91      0.91      0.91     12500
weighted avg       0.91      0.91      0.91     12500

Confusion Matrix: 
 [[5478  679]
 [ 497 5846]]
Accuracy with C = 1: 90.59%
f1-score : 
90.86%


### Phân tích cải tiến 1
- Khi tăng giá trị C lên (C = 1), giá trị siêu tham số C này có thể được coi là một giá trị trung bình, giúp cân bằng giữa việc xem xét điểm dữ liệu sai lệch và tối ưu hóa độ chính xác trên tập huấn luyện.
Kết quả cho thấy một mô hình khá tốt trên dữ liệu huấn luyện và có khả năng tổng quát hóa tốt trên dữ liệu kiểm tra. Chỉ số chính xác cùng với những chỉ số khác cũng cao hơn so với trước.

Vậy nếu khi giảm giá trị siêu tham số này xuống thì kết quả sẽ như thế nào!


### C = 0.1

In [73]:
LSVC = LinearSVC(C=0.1, random_state=42)
LSVC.fit(tfidf_vecto_train, y_train)
du_doan = LSVC.predict(tfidf_vecto_test)

In [74]:
print("Classification report: \n",classification_report(y_test,du_doan, target_names = ['Positive','Negative']))
print("Confusion Matrix: \n",confusion_matrix(y_test,du_doan))
dochinhxac = metrics.accuracy_score(y_test,du_doan)
print("Accuracy with C = 0.1: \n" +str('{:04.2f}'.format(dochinhxac *100)) + "%")

f1 = metrics.f1_score(y_test,du_doan)
print("f1-score : \n"+str('{:04.2f}'.format(f1 *100)) + "%")

Classification report: 
               precision    recall  f1-score   support

    Positive       0.90      0.87      0.88      6157
    Negative       0.88      0.90      0.89      6343

    accuracy                           0.89     12500
   macro avg       0.89      0.89      0.89     12500
weighted avg       0.89      0.89      0.89     12500

Confusion Matrix: 
 [[5357  800]
 [ 621 5722]]
Accuracy with C = 0.1: 
88.63%
f1-score : 
88.95%


### Phân tích cải tiến 2
- Khi điều chỉnh giảm giá trị C = 0.1, đồng nghĩa với việc mô hình đặt nhiều hạn chế hơn cho việc xem xét dữ liệu sai lệch.
Khi giá trị C nhỏ, mô hình có thể trở nên quá nghiêm ngặt, tạo ra một siêu phẳng quá đơn giản.
Điều này có thể dẫn đến underfitting, vì mô hình không thể nắm bắt đủ thông tin trong dữ liệu. Vì vậy chỉ số accuracy khi điều chỉnh C = 0.1 là thấp hơn so với trước.

Vậy nếu tăng giá trị C này lên cao có thể tăng thêm tỉ lệ chính xác của mô hình hay không?


### C = 10

In [75]:
LSVC = LinearSVC(C=10, random_state=42)
LSVC.fit(tfidf_vecto_train, y_train)
du_doan = LSVC.predict(tfidf_vecto_test)

In [76]:
print("Classification report: \n",classification_report(y_test,du_doan, target_names = ['Positive','Negative']))
print("Confusion Matrix: \n",confusion_matrix(y_test,du_doan))
dochinhxac = metrics.accuracy_score(y_test,du_doan)
print("Accuracy with C = 10: \n" +str('{:04.2f}'.format(dochinhxac *100)) + "%")

f1 = metrics.f1_score(y_test,du_doan)
print("f1-score : \n"+str('{:04.2f}'.format(f1 *100)) + "%")

Classification report: 
               precision    recall  f1-score   support

    Positive       0.92      0.89      0.91      6157
    Negative       0.90      0.92      0.91      6343

    accuracy                           0.91     12500
   macro avg       0.91      0.91      0.91     12500
weighted avg       0.91      0.91      0.91     12500

Confusion Matrix: 
 [[5494  663]
 [ 488 5855]]
Accuracy with C = 10: 
90.79%
f1-score : 
91.05%


### Phân tích cải tiến 3
- Khi tăng giá trị C = 10, mô hình trở nên ít nghiêm ngặt hơn đối với điểm dữ liệu sai lệch. Mặc dù đúng là chỉ số accuracy có cao hơn một chút đối với C = 1 nhưng điều này có thể dẫn đến mô hình có khả năng bị overfitting, vì nó cố gắng tối ưu hóa độ chính xác trên tập huấn luyện một cách quá mức.

Sự cải thiện độ chính xác khi tăng giá trị của C có thể xuất phát từ việc mô hình trở nên có khả năng học một cách tốt hơn trên dữ liệu huấn luyện, nhưng điều này cũng có thể gây ra overfitting trên tập huấn luyện và không tổng quát hóa tốt trên dữ liệu kiểm tra.

# Sử dụng GridSearchCV

In [77]:
from sklearn.model_selection import GridSearchCV

# Bước 1: Tinh chỉnh siêu tham số
param_grid = {'C': [0.1, 0.5, 1, 10]}
grid_search = GridSearchCV(LinearSVC(), param_grid, cv=5)
grid_search.fit(tfidf_vecto_train, y_train)
# Bước 2: Lấy ra siêu tham số tốt nhất
best_params = grid_search.best_params_
print("Siêu tham số tốt nhất: ",best_params)

Siêu tham số tốt nhất:  {'C': 10}


GridSearchCV giúp tìm kiếm siêu tham số C tốt nhất một cách nhanh hơn, tối ưu hơn

# Sử dụng RandomizedSearchCV

In [78]:
from sklearn.model_selection import RandomizedSearchCV

# Xác định phạm vi cho tham số C
param_dist = {'C': np.linspace(-1, 10, 50)}
# Tạo Random Search với phạm vi giá trị C và mô hình
random_search = RandomizedSearchCV(LinearSVC(), param_dist, n_iter=10, cv=5)  # cv là số lần cross-validation
# Lấy giá trị tốt nhất cho tham số C
random_search.fit(tfidf_vecto_train, y_train)
random_search.best_params_

best_params = random_search.best_params_
print("Siêu tham số tốt nhất: ",best_params)
# Bước 3: Tạo mô hình Linear SVC với siêu tham số tốt nhất và huấn luyện
LSVC = LinearSVC(C=best_params['C'])
LSVC.fit(tfidf_vecto_train, y_train)
du_doan = LSVC.predict(tfidf_vecto_test)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
10 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py", line 261, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_par

Siêu tham số tốt nhất:  {'C': 8.877551020408163}


In [79]:
print("Classification report: \n",classification_report(y_test,du_doan, target_names = ['Positive','Negative']))
print("Confusion Matrix: \n",confusion_matrix(y_test,du_doan))
dochinhxac = metrics.accuracy_score(y_test,du_doan)
print("Độ chính xác: \n" +str('{:04.2f}'.format(dochinhxac *100)) + "%")

f1 = metrics.f1_score(y_test,du_doan)
print("f1 : \n"+str('{:04.2f}'.format(f1 *100)) + "%")

Classification report: 
               precision    recall  f1-score   support

    Positive       0.92      0.89      0.91      6157
    Negative       0.90      0.92      0.91      6343

    accuracy                           0.91     12500
   macro avg       0.91      0.91      0.91     12500
weighted avg       0.91      0.91      0.91     12500

Confusion Matrix: 
 [[5494  663]
 [ 487 5856]]
Độ chính xác: 
90.80%
f1 : 
91.06%


RandomSearch có thể kiểm tra qua một loạt giá trị để tìm giá trị tốt nhất, tối ưu nhất cho siêu tham số để sử dụng cho mô hình.